# Taller David Salamanca

### Funciones básicas: `sumaCuadrados`, `normaVector`

Estas funciones calculan el valor absoluto al cuadrado de un número complejo y la norma de un vector, respectivamente. Están correctamente implementadas, aunque se pueden optimizar con operaciones nativas de `NumPy`.

### Función `superposition`

Calcula la probabilidad de encontrar un sistema en una posición específica dada una superposición cuántica. Está bien implementada y utiliza `sumaCuadrados` y `normaVector` para obtener la norma y la amplitud.

### Función `transition`

Calcula la amplitud de transición entre dos vectores cuánticos utilizando el producto interno y las normas de ambos vectores. Funciona adecuadamente, pero se podría hacer uso del producto interno de `NumPy` de manera más directa.

### Funciones para matrices: `tranpuestaMatriz`, `conjugadaMatriz`, `adjuntaMatriz`

Estas funciones calculan la transpuesta y la adjunta de una matriz, pero la implementación de `tranpuestaMatriz` puede simplificarse usando `NumPy` directamente. Además, `adjuntaMatriz` es más eficiente usando `numpy.transpose` y `numpy.conjugate`.

### Función `probarHermitiana`

Verifica si una matriz es hermitiana (igual a su adjunta conjugada). Sin embargo, hay una duplicidad con la función `proobarHermitiana`. Se podría consolidar y mejorar la estructura de la verificación.

### Funciones de observables: `media`, `varianza`

Estas funciones calculan la media y la varianza de un observable dado un vector de estado. El cálculo es correcto, pero puede mejorarse con manejo más eficiente de matrices y vectores. La función `varianza` parece estar implementada adecuadamente usando las operaciones de matrices.

### Propios valores y transiciones: `valPropio`, `probabilidadTransicion`

Estas funciones calculan los valores propios de un observable y la probabilidad de que el sistema cuántico transite a un vector propio tras la observación. El uso de `numpy.linalg.eig` es correcto para el cálculo de los valores propios.

In [34]:
import math
import numpy as np

def sumaCuadrados(c):
    return abs(c) ** 2

def normaVector(v):
    return np.sqrt(np.sum(np.abs(v) ** 2))

def superposition(vec, pos):
    c = sumaCuadrados(vec[pos])
    norma = normaVector(vec)
    return round(c / norma ** 2, 4)

def transition(vec1, vec2):
    productoInterno = np.vdot(vec1, vec2)
    normaVec1 = normaVector(vec1)
    normaVec2 = normaVector(vec2)
    return np.round(productoInterno / (normaVec1 * normaVec2), 4)

def adjuntaMatriz(mat):
    return np.conjugate(np.transpose(mat))

def probarHermitiana(mat):
    return np.array_equal(mat, adjuntaMatriz(mat))

def accionMatrizVector(v, mat):
    return np.dot(mat, v)

def innerProduct(vec1, vec2):
    return np.vdot(vec1, vec2)

def media(observable, vecEstado):
    if probarHermitiana(observable):
        prod = accionMatrizVector(vecEstado, observable)
        return innerProduct(prod, vecEstado).real
    else:
        raise ValueError("La matriz no es Hermitiana")

def varianza(observable, vecEstado):
    mediaC = media(observable, vecEstado)
    identidad = np.eye(len(observable)) * mediaC
    resta = observable - identidad
    producto = np.dot(resta, resta)
    return media(producto, vecEstado)

def valPropio(mat):
    valores, vectores = np.linalg.eig(mat)
    return valores, vectores

def probabilidadTransicion(observable, posicion):
    valoresPropios, _ = valPropio(observable)
    return superposition(valoresPropios, posicion)

def unitaria(mat):
    identidad = np.eye(mat.shape[0])
    return np.allclose(identidad, np.dot(mat.conj().T, mat))

### Ejercicio 4.3.1
Calcula todos los posibles estados en los que el sistema descrito en el **Ejercicio 4.2.2** puede transitar después de realizar una medición.

### Ejercicio 4.3.2
Realiza los mismos cálculos que en el Ejercicio 4.3.1, pero luego dibuja la distribución de probabilidad de los valores propios como en el ejemplo anterior.

### Ejercicio 4.4.1
Verifica que dos matrices dadas son unitarias. Luego multiplícalas y verifica que su producto también sea una matriz unitaria.

### Ejercicio 4.4.2
Regresa al Ejemplo 3.3.2 del billar cuántico, mantén el mismo vector de estado inicial [1, 0, 0, 0]T, pero cambia el mapa unitario para observar el nuevo estado resultante.

In [43]:
def ejercicio_431(observable):
    
    spinX = np.array([1, 0])
    prod = np.dot(observable, spinX)
    probabilidad = superposition(prod, 0)
    return probabilidad


observable = np.array([[0, 1/2], [1/2, 0]])
print("Ejercicio 431:", ejercicio_431(observable))


def ejercicio_432(observable):
    
    spinX = np.array([1, 0])
    valores_propios, vectores_propios = np.linalg.eig(observable)
    producto = np.dot(observable, spinX)
    
    p1 = normaVector(np.dot(producto, vectores_propios[:, 0]))
    p2 = normaVector(np.dot(producto, vectores_propios[:, 1]))

    resultado = p1 * valores_propios[0] + p2 * valores_propios[1]
    return resultado

print("Ejercicio 432:", ejercicio_432(observable))


def ejercicio_441(u1, u2):
    
    u1 = np.array(u1)
    u2 = np.array(u2)
    
    isU1 = np.allclose(np.dot(u1.conj().T, u1), np.eye(u1.shape[0]))
    isU2 = np.allclose(np.dot(u2.conj().T, u2), np.eye(u2.shape[0]))
    
    producto = np.dot(u1, u2)
    productoUni = np.allclose(np.dot(producto.conj().T, producto), np.eye(producto.shape[0]))
    
    return isU1, isU2, productoUni


u1 = [[0, 1], [1, 0]]
u2 = [[math.sqrt(2) / 2, math.sqrt(2) / 2], [math.sqrt(2) / 2, -(math.sqrt(2) / 2)]]
print("Ejercicio 441:", ejercicio_441(u1, u2))


def ejercicio_442(estadoInicial, mat):
    
    estadoInicial = np.array(estadoInicial)
    matriz = np.array(mat)
    estado = np.dot(matriz, estadoInicial)
    
    probabilidad = probabilidadTransicion(estado, 1)
    return probabilidad


unitaryMap = [[[0, 1 / math.sqrt(2), 1 / math.sqrt(2), 0], 
              [(1 / math.sqrt(2)) * 1j, 0, 0, 1 / math.sqrt(2)]], 
              [[1 / math.sqrt(2), 0, 0, (1 / math.sqrt(2)) * 1j], 
              [0, 1 / math.sqrt(2), -(1 / math.sqrt(2)), 0]]]

initialVector = [1, 0, 0, 0]
print("Ejercicio 442:", ejercicio_442(initialVector, unitaryMap))

Ejercicio 431: 0.0
Ejercicio 432: 0.0
Ejercicio 441: (True, True, True)
Ejercicio 442: 0.5
